<a href="https://colab.research.google.com/github/ak2742/mlplay/blob/Fine-Tuning/05)_CSV_RAG_with_gradio_interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Code to mount Google Drive at Colab Notebook instance
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Workaround to avoid following error at notebook
# NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
#@title Install all required libraries

# Huggingface libraries to run LLM.
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes
!pip install -q -U huggingface_hub

#LangChain related libraries
!pip install -q -U langchain==0.1.2

#Open-source pure-python PDF library capable of splitting, merging, cropping,
#and transforming the pages of PDF files
!pip install -q -U pypdf

#Python framework for state-of-the-art sentence, text and image embeddings.
!pip install -q -U sentence-transformers

# FAISS Vector Databses specific Libraries
!pip install -q -U faiss-gpu

!pip install -q -U gradio

In [ ]:
#@title imports and Check for GPU

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig
import torch
from langchain.llms import HuggingFacePipeline

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain


device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))

In [ ]:
#@title Create Model

origin_model_path = "mistralai/Mistral-7B-Instruct-v0.1"
model_path = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
bnb_config = BitsAndBytesConfig \
              (
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
              )
model = AutoModelForCausalLM.from_pretrained (model_path, trust_remote_code=True,
                                              quantization_config=bnb_config,
                                              device_map="auto")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(origin_model_path)

In [ ]:
#@title Creating pipelines to run LLM at Colab notebook

text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=300,
    temperature = 0.3,
    do_sample=True,
)
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
#@title Fn to create database and Conversational Retrieval Chain

from langchain.vectorstores import FAISS

def create_qa_chain(chunked_docs, search_kwargs=4, return_source_documents=False):
    # Using HuggingFace embeddings
    embeddings = HuggingFaceEmbeddings()

    # Create FAISS vector database
    db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

    # Connect query to FAISS index using a retriever
    retriever = db.as_retriever(
        search_type="similarity",
        search_kwargs={'k': search_kwargs}
    )

    # Create the Conversational Retrieval Chain
    qa_chain = ConversationalRetrievalChain.from_llm(mistral_llm, retriever, return_source_documents)

    return qa_chain

# **Approach 1**

In [ ]:
#@title Load CSV with langchain CSVLoader

from langchain_community.document_loaders.csv_loader import CSVLoader
# Load the CSV file
file_path = '/content/sample_data/california_housing_test.csv'
loader = CSVLoader(file_path)
data = loader.load()

# Split the documents into smaller chunks
#separator=""
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunked_docs  = text_splitter.split_documents(data)

# Create the Conversational Retrieval Chain
qa_chain = create_qa_chain(chunked_docs)

# **Approach 2**

In [ ]:
!pip install -q -U pandas

In [ ]:
#@title Load CSV with Pandas

import pandas as pd
# Load the CSV file
df = pd.read_csv(file_path)

In [ ]:
from langchain.schema.document import Document
text = ""

#append CSV data to text
for ind in df.index:
    text += f"{df['longitude'][ind]}  {df['latitude'][ind]}  {df['population'][ind]}\n#####\n"
#Converting text to LangChain documents so that StuffDocumentsChain can understand Input
documents = Document(page_content=text, metadata={"source": "local"})

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Split the documents into smaller chunks
#separators=["\n\n", "\n", " ", ""]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators="[#####]")
chunked_docs  = text_splitter.split_documents([documents])

# Create the Conversational Retrieval Chain
qa_chain = create_qa_chain(chunked_docs)

# **Add Gradio UI**

In [ ]:
import gradio as gr

def gradio_fn(msg, chat_history):
    result = qa_chain.invoke({'question': msg, 'chat_history': chat_history})
    history.append((msg, result['answer']))
    return result['answer']

gr.ChatInterface(
    gradio_fn,
    examples=[["which area has the highest population"]],
    chatbot=gr.Chatbot(height=300),
    title="CSV QA",
    textbox=gr.Textbox(placeholder="Ask your question here", container=False, scale=7),
    theme="soft",
    description="Ask me any question on the CSV data",
    cache_examples=True,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
    ).launch(share=True, debug=True)